In [217]:
import pandas as pd 
import numpy as np
from settings import GenerateShiftsData
import math
import os
from datetime import datetime
import json
import random


In [226]:
AmountOfShiftsRequested = {
    'Elran' : [4],
    'Ben' : [3,4],
    'Alon':[5],
    'Roy':[4],
    'Merav':[2],
    'Ofek':[3,4],
    'Benjamin':[3,4]
}
data = GenerateShiftsData()
shifts = data.getShifts()
workers = data.getWorkers()
workersShifts = data.getWorkersShifts()




OBJECTING THE DATA

In [219]:
randomNumber = random.randint(5,15)
print(randomNumber)
randomNumber2 = random.randint(1,5)
print(randomNumber2)


11
5


In [220]:
def getShiftsByWorker(worker):
    return workersShifts[worker]

def getWorkersByShift(shift):
    output = []
    for i in workers:
        if getShiftsByWorker(i)[shift] == 1:
            output.append(i)
    return output

def getWorkersByShiftAs2(shift):
    output = []
    for i in workers:
        if getShiftsByWorker(i)[shift] == 2:
            output.append(i)
    return output

def getWorkersByShiftAs3(shift):
    output = []
    for i in workers:
        if getShiftsByWorker(i)[shift] == 3:
            output.append(i)
    return output

def getShiftsInNum(shift):
    output = {}
    for i in shifts:
        output[i] = shifts.index(i)
    return output


def GetNightShifts():
    output = []
    for i in shifts:
        if 'Night' in i:
            output.append(i)
    return output

def getMorningShifts():
    output = []
    for i in shifts:
        if 'Morning' in i:
            output.append(i)
    return output

def getEveningShifts():
    output = []
    for i in shifts:
        if 'Evening' in i:
            output.append(i)
    return output

def amountOfMorningShiftsRequestedAs3(worker):
    amount = 0
    for j in getMorningShifts():
        if workersShifts[worker][j] == 3:
            amount += 1
    return amount

def amountOfEveningShiftsRequestedAs3(worker):
    amount = 0
    for j in getEveningShifts():
        if workersShifts[worker][j] == 3:
            amount += 1 
    return amount

def amountOfMorningShiftsRequestedAs2(worker):
    amount = 0
    for j in getMorningShifts():
        if workersShifts[worker][j] == 2:
            amount += 1
    return amount

def amountOfEveningShiftsRequestedAs2(worker):
    amount = 0
    for j in getEveningShifts():
        if workersShifts[worker][j] == 2:
            amount += 1
    return amount

def getAmountOfShiftsByWorker(worker):
    if len(AmountOfShiftsRequested[worker]) == 1:
        return AmountOfShiftsRequested[worker][0]
    else:
        avg = sum(AmountOfShiftsRequested[worker])/len(AmountOfShiftsRequested[worker])
        return math.ceil(avg)

def createEmployeeScoreDict():
    scoreDict = {}
    for i in workers:
        scoreDict[i] ={}
        scoreDict[i]['night_shifts_until_shift'] = 0
        scoreDict[i]['max_night_shifts'] = 2
        scoreDict[i]['max_shifts'] = getAmountOfShiftsByWorker(i)
        scoreDict[i]['shifts_until_shift'] = 0
        scoreDict[i]['last_shift'] = 0
        scoreDict[i]['last_night_shift'] = 0
        scoreDict[i]['amount_of_morning_shifts_as_3'] = amountOfMorningShiftsRequestedAs3(i)
        scoreDict[i]['amount_of_morning_shifts_got_as_3'] = 0
        scoreDict[i]['amount_of_evening_shifts_as_3'] = amountOfEveningShiftsRequestedAs3(i)
        scoreDict[i]['amount_of_evening_shifts_got_as_3'] = 0
        scoreDict[i]['amount_of_morning_shifts_as_2'] = amountOfMorningShiftsRequestedAs2(i)
        scoreDict[i]['amount_of_morning_shifts_got_as_2'] = 0
        scoreDict[i]['amount_of_evening_shifts_as_2'] = amountOfEveningShiftsRequestedAs2(i)
        scoreDict[i]['amount_of_evening_shifts_got_as_2'] = 0
    return scoreDict





OBJECTS

In [221]:
EMPLOYEES_SCORE_DICT = createEmployeeScoreDict()
NIGHT_SHIFTS = GetNightShifts()
SHIFTS_TO_NUM = getShiftsInNum(shifts)
MORNING_SHIFTS = getMorningShifts()
EVENING_SHIFTS = getEveningShifts()


CALCULATION PART

In [222]:

def calcScoreOnLatestShift(shift,worker):
    score = 0 
    lastShift = EMPLOYEES_SCORE_DICT[worker]['last_shift']
    currentShift = SHIFTS_TO_NUM[shift]
    score += (currentShift - lastShift) 
    return score 

def calcScoreOnMaxShifts(shift,worker):
    score = 0

    maxShifts = EMPLOYEES_SCORE_DICT[worker]['max_shifts']
    shiftsUntilShift = EMPLOYEES_SCORE_DICT[worker]['shifts_until_shift']

    score += (maxShifts - shiftsUntilShift) + random.randint(1, 10)
    return score

def calcScoreOnShiftAfterShift(shift,worker):
    score = 0
    shiftNum = SHIFTS_TO_NUM[shift]
    lastShift = EMPLOYEES_SCORE_DICT[worker]['last_shift']
    if shiftNum - lastShift == 1:
        score -= 10
    return score

def lowerScoreOnShiftsAs1(shift,worker):
    score = 0
    if workersShifts[worker][shift] == 1:
        score -= 10
    return score

def calcScoreOnLatestNightShift(shift,worker):
    score = 0
    lastNightShift = EMPLOYEES_SCORE_DICT[worker]['last_night_shift']
    currentShift = SHIFTS_TO_NUM[shift]

    score += (currentShift - lastNightShift) + random.randint(1, 10)

    return score * 2

def calcScoreOnMaxNightShifts(shift,worker):
    score = 0
    maxNightShifts = EMPLOYEES_SCORE_DICT[worker]['max_night_shifts']
    nightShiftsUntilShift = EMPLOYEES_SCORE_DICT[worker]['night_shifts_until_shift']
    maxShifts = EMPLOYEES_SCORE_DICT[worker]['max_shifts']
    shiftsUntilShift = EMPLOYEES_SCORE_DICT[worker]['shifts_until_shift']

    score += (maxNightShifts - nightShiftsUntilShift)
    score += (maxShifts - shiftsUntilShift) + random.randint(1, 10)
    return score * 1.5 



def calcScoreOnSelectedShift(shift,worker):
    score = 0
    if workersShifts[worker][shift] == 3:
        score += (3 *3) +randomNumber + random.randint(1, 10)
    elif workersShifts[worker][shift] == 2:
        score += (2 *1.5) + random.randint(1, 10) 

    return score





In [ ]:
## CONSTRAINTS

def addConstraintOnMaxShifts(shift,worker):
    maxShifts = EMPLOYEES_SCORE_DICT[worker]['max_shifts']
    shiftsUntilShift = EMPLOYEES_SCORE_DICT[worker]['shifts_until_shift']
    if shiftsUntilShift >= maxShifts:
        return False
    else:
        return True

def addConstraintOnMaxNightShifts(shift,worker):
    maxNightShifts = EMPLOYEES_SCORE_DICT[worker]['max_night_shifts']
    nightShiftsUntilShift = EMPLOYEES_SCORE_DICT[worker]['night_shifts_until_shift']
    if nightShiftsUntilShift >= maxNightShifts:
        return False
    else:
        return True

def addConstraintOnShiftAfterShift(shift,worker):
    shiftNum = SHIFTS_TO_NUM[shift]
    lastShift = EMPLOYEES_SCORE_DICT[worker]['last_shift']
    if shiftNum - lastShift == 1:
        return False
    else:
        return True


In [223]:
import shutil
currentTime = datetime.now()
def calcScoreOnShift(shift,worker):
    score = {}
    score['latest_shift'] = calcScoreOnLatestShift(shift,worker)
    score['max_shifts'] = calcScoreOnMaxShifts(shift,worker)
    score['latest_night_shift'] = calcScoreOnLatestNightShift(shift,worker)
    score['max_night_shifts'] = calcScoreOnMaxNightShifts(shift,worker)
    score['selected_shift'] = calcScoreOnSelectedShift(shift,worker)
    score['total_score'] = sum(score.values())
    return score

def findWorker(shift,workers):
    scoreDict = {}
    all_scoreDict = {}
    highestScore = 0
    selectedWorker = ''
    for i in workers:
        scoreDict[i] = calcScoreOnShift(shift,i)
        all_scoreDict[i] = scoreDict[i]
        score = scoreDict[i]['total_score']
        if score > highestScore:
            highestScore = score
            selectedWorker = i
    return ({'selected_worker':selectedWorker,'score':highestScore,'score_dict':scoreDict,'all_score_dict':all_scoreDict})

def findWorkerForShift(shift):
    workers = getWorkersForShifts(shift)
    return findWorker(shift,workers)


def getWorkersForShifts(shifts):
    schedule = {}
    for i in shifts:
        workersSelectedAs3 = getWorkersByShiftAs3(i)
        workersSelectedAs2 = getWorkersByShiftAs2(i)
        print(workersSelectedAs3)
        responseFromFindWorker = findWorker(i,workers)
        selectedWorker = responseFromFindWorker['selected_worker']
        schedule[i] = selectedWorker
        #schedule[i]['selected_worker'] = selectedWorker
        #schedule[i]['all_score_dict'] = responseFromFindWorker['all_score_dict']
        if selectedWorker in workersSelectedAs3:
            EMPLOYEES_SCORE_DICT[selectedWorker]['last_shift'] = SHIFTS_TO_NUM[i]
            EMPLOYEES_SCORE_DICT[selectedWorker]['shifts_until_shift'] += 1
            if i in MORNING_SHIFTS:
                EMPLOYEES_SCORE_DICT[selectedWorker]['amount_of_morning_shifts_got_as_3'] += 1
            elif i in EVENING_SHIFTS:
                EMPLOYEES_SCORE_DICT[selectedWorker]['amount_of_evening_shifts_got_as_3'] += 1
            elif i in NIGHT_SHIFTS:
                EMPLOYEES_SCORE_DICT[selectedWorker]['last_night_shift'] = SHIFTS_TO_NUM[i]
                EMPLOYEES_SCORE_DICT[selectedWorker]['night_shifts_until_shift'] += 1
        elif selectedWorker in workersSelectedAs2:
            EMPLOYEES_SCORE_DICT[selectedWorker]['last_shift'] = SHIFTS_TO_NUM[i]
            EMPLOYEES_SCORE_DICT[selectedWorker]['shifts_until_shift'] += 1
            EMPLOYEES_SCORE_DICT[selectedWorker]['amount_of_morning_shifts_got_as_2'] += 1
            EMPLOYEES_SCORE_DICT[selectedWorker]['amount_of_evening_shifts_got_as_2'] += 1
            if i in NIGHT_SHIFTS:
                EMPLOYEES_SCORE_DICT[selectedWorker]['last_night_shift'] = SHIFTS_TO_NUM[i]
                EMPLOYEES_SCORE_DICT[selectedWorker]['night_shifts_until_shift'] += 1
        else:
            EMPLOYEES_SCORE_DICT[selectedWorker]['last_shift'] = SHIFTS_TO_NUM[i]
            EMPLOYEES_SCORE_DICT[selectedWorker]['shifts_until_shift'] = 1
            if i in NIGHT_SHIFTS:
                EMPLOYEES_SCORE_DICT[selectedWorker]['last_night_shift'] = SHIFTS_TO_NUM[i]
                EMPLOYEES_SCORE_DICT[selectedWorker]['night_shifts_until_shift'] = 1



    return schedule


schedule1 = getWorkersForShifts(shifts)
print(schedule1)

def getScoreForSchedule(schedule12):
    ##createOuputof the schedule
    time = datetime.now()
    outputTotal = {}
    outputIndividual = {}
    ## create folder for output
    if not os.path.exists(f'output {str(time)}'):
        os.makedirs(f'output {str(time)}')
    with open(f'output {str(time)}.json', 'w') as outfile:
        json.dump(schedule12, outfile)
    totalScore = 0
    maxMinusCurrent = 0
    amountOfNightShifts = 0
    for i in workers:
        outputIndividual[i] = {}
        outputIndividual[i]['max_minus_current'] = EMPLOYEES_SCORE_DICT[i]['max_shifts'] - EMPLOYEES_SCORE_DICT[i]['shifts_until_shift']
        outputIndividual[i]['amount_of_night_shifts'] = EMPLOYEES_SCORE_DICT[i]['night_shifts_until_shift']
        outputIndividual[i]['amount_of_morning_shifts_got_gap_3'] = EMPLOYEES_SCORE_DICT[i]['amount_of_morning_shifts_as_3'] - EMPLOYEES_SCORE_DICT[i]['amount_of_morning_shifts_got_as_3']
        outputIndividual[i]['amount_of_evening_shifts_got_gap_3'] = EMPLOYEES_SCORE_DICT[i]['amount_of_evening_shifts_as_3'] - EMPLOYEES_SCORE_DICT[i]['amount_of_evening_shifts_got_as_3']
        outputIndividual[i]['amount_of_morning_shifts_got_gap_2'] = EMPLOYEES_SCORE_DICT[i]['amount_of_morning_shifts_as_2'] - EMPLOYEES_SCORE_DICT[i]['amount_of_morning_shifts_got_as_2']
        outputIndividual[i]['amount_of_evening_shifts_got_gap_2'] = EMPLOYEES_SCORE_DICT[i]['amount_of_evening_shifts_as_2'] - EMPLOYEES_SCORE_DICT[i]['amount_of_evening_shifts_got_as_2']
        totalScore += outputIndividual[i]['max_minus_current']
        maxMinusCurrent += outputIndividual[i]['max_minus_current']
        amountOfNightShifts += outputIndividual[i]['amount_of_night_shifts']
    outputTotal['total_score'] = totalScore
    print("Total Score: ",totalScore)
    outputTotal['max_minus_current'] = maxMinusCurrent
    outputTotal['amount_of_night_shifts'] = amountOfNightShifts
    with open(f'output {str(time)} total.json {totalScore}', 'w') as outfile:
        json.dump(outputTotal, outfile)
    with open(f'output {str(time)} individual.json', 'w') as outfile:
        json.dump(outputIndividual, outfile)
    ## move the JSON to the folder
    shutil.move(f'output {str(time)}.json', f'output {str(time)}/output {str(time)}.json')
    shutil.move(f'output {str(time)} total.json {totalScore}', f'output {str(time)}/output {str(time)} total.json')
    shutil.move(f'output {str(time)} individual.json', f'output {str(time)}/output {str(time)} individual.json')
    return outputTotal
    
        
    




getScoreForSchedule(schedule1)

    







        
    




['Elran', 'Ofek', 'Benjamin']
['Elran', 'Alon', 'Merav', 'Ofek']
['Elran', 'Ben', 'Roy', 'Benjamin']
['Elran', 'Roy', 'Ofek']
['Elran', 'Merav', 'Benjamin']
['Ben', 'Roy']
['Ofek']
['Elran', 'Ben', 'Benjamin']
['Elran', 'Alon', 'Roy', 'Merav', 'Ofek']
['Roy', 'Benjamin']
['Elran', 'Alon']
['Elran', 'Benjamin']
['Elran', 'Ben', 'Merav', 'Ofek']
['Alon', 'Ofek']
['Roy']
['Alon']
['Alon', 'Ofek', 'Benjamin']
['Elran', 'Ben', 'Ofek']
['Elran', 'Benjamin']
['Ben', 'Alon', 'Roy']
['Elran']
{'Sunday Morning': 'Ofek', 'Sunday Evening': 'Elran', 'Sunday Night': 'Roy', 'Monday Morning': 'Ofek', 'Monday Evening': 'Benjamin', 'Monday Night': 'Ben', 'Tuesday Morning': 'Alon', 'Tuesday Evening': 'Elran', 'Tuesday Night': 'Merav', 'Wednesday Morning': 'Roy', 'Wednesday Evening': 'Alon', 'Wednesday Night': 'Elran', 'Thursday Morning': 'Ofek', 'Thursday Evening': 'Ofek', 'Thursday Night': 'Roy', 'Friday Morning': 'Alon', 'Friday Evening': 'Benjamin', 'Friday Night': 'Ben', 'Saturday Morning': 'Benjamin

{'total_score': 6, 'max_minus_current': 6, 'amount_of_night_shifts': 7}

EXAMPLES